<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/4_analysis_hate_ar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter

## Analisis del Dataset completo marzo 2020 - abril 2021 de X

**Natalia Dedandi**

Se aplica el clasificador seleccionado a todo el conjunto de datos recoelctado para analizar las características del odio en Argentina durante la pandemia

In [1]:
!pip install datasets seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 req

Recupero el conjutno completo de datos: https://huggingface.co/datasets/piuba-bigdata/articles_and_comments



In [2]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("piubamas/articles_and_comments")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/537201 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tweet_id', 'text', 'title', 'url', 'user', 'body', 'created_at', 'comments'],
        num_rows: 537201
    })
})

In [4]:
ds1 = dataset["train"]

Ejemplo de una fila

In [5]:
ds1[122]

{'tweet_id': '1376979520729849858',
 'text': 'Una fuerte apuesta a la rueda de la agroindustrialización argentina https://t.co/1o6ThCdtxw https://t.co/TLfGhLBjHu',
 'title': 'Una fuerte apuesta a la rueda de la agroindustrialización argentina',
 'url': 'https://www.lanacion.com.ar/economia/campo/una-fuerte-apuesta-a-la-rueda-de-la-agroindustrializacion-argentina-nid30032021/?utm_term=Autofeed&utm_medium=Echobox&utm_source=Twitter#Echobox=1617130198',
 'user': 'LANACION',
 'body': 'Desde el primer día, YPF Agro se propuso proveer al campo argentino de los insumos y las energías necesarias para producir más y mejor, en un contexto donde la demanda de alimentos para la población mundial sigue creciendo.\n\nEl negocio de agro de YPF cerró 2020 con ventas mayores a 1.428 millones de dólares y más de 1,5 millones de toneladas de grano canjeadas, y apuesta a incrementar estos resultados en 2021.\n\nEn ese sentido, lanzó su nueva campaña de comunicación orientada a reforzar la idea de circular

En este conjunto de datos tengo las etiquetas de la clasificación realizada con el modelo BETO:

https://huggingface.co/piuba-bigdata/beto-contextualized-hate-speech

Puede usarse para comparar pero en principio nos interesa recuperar los comentarios completos para aplicarle el clasificador seleccionado

In [6]:
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from collections import Counter

In [7]:
tweets_arg = []

for noticia in tqdm(dataset['train']):

    date_noticia = noticia["created_at"]
    # Convertimos la fecha de la noticia a un objeto de python
    datenoticia = datetime.strptime(date_noticia, "%Y-%m-%dT%H:%M:%S%fZ")

    for comment in noticia["comments"]:
        date_comment = comment["created_at"]
        # Convertimos la fecha del comentario a un objeto de python
        datecomment = datetime.strptime(date_comment, "%Y-%m-%dT%H:%M:%S%fZ")
        # anexa comentarios de diarios argentinos
        tweets_arg.append({
            "tweet_id_noticia": noticia["tweet_id"],
            #"user_noticia": noticia["user"],
           # "date_noticia": datenoticia,
           # "title_noticia": noticia["title"],
            #"url_noticia": noticia["url"],
            "tweet": comment["text"],
            "date_tweet": datecomment,
            "APPEARANCE":comment["prediction"]["APPEARANCE"],
            "CLASS":comment["prediction"]["CLASS"],
            "CRIMINAL":comment["prediction"]["CRIMINAL"],
            "DISABLED":comment["prediction"]["DISABLED"],
            "LGBTI":comment["prediction"]["LGBTI"],
            "POLITICS":comment["prediction"]["POLITICS"],
            "RACISM":comment["prediction"]["RACISM"],
            "WOMEN":comment["prediction"]["WOMEN"],
            "CALLS":comment["prediction"]["CALLS"]
            })

  0%|          | 0/537201 [00:00<?, ?it/s]

In [10]:
tweets_arg[123]

{'tweet_id_noticia': '1376941040205238272',
 'tweet': '@elmundoes Nazis. Son nazis.',
 'date_tweet': datetime.datetime(2021, 3, 30, 17, 14, 4, 100000),
 'APPEARANCE': 0,
 'CLASS': 0,
 'CRIMINAL': 0,
 'DISABLED': 0,
 'LGBTI': 0,
 'POLITICS': 1,
 'RACISM': 0,
 'WOMEN': 0,
 'CALLS': 0}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Creo un archivo csv con estos datos para tenerlos por las dudas

In [9]:
# prompt: save to csv file tweets_arg
#
#import csv

#keys = tweets_arg[0].keys()

#with open('tweets_arg.csv', 'w', newline='') as output_file:
#    dict_writer = csv.DictWriter(output_file, keys)
#    dict_writer.writeheader()
#    dict_writer.writerows(tweets_arg)


In [1]:
# prompt: creare a dataframe of a sample od tweets_arg

import random
# Sample 1000 tweets
sample_tweets = random.sample(tweets_arg, 10)

# Create dataframe
df_sample = pd.DataFrame(sample_tweets)


NameError: name 'tweets_arg' is not defined

In [15]:
# prompt: show the 5 first df_sample

df_sample.head()


,tweet_id_noticia,tweet,date_tweet,APPEARANCE,CLASS,CRIMINAL,DISABLED,LGBTI,POLITICS,RACISM,WOMEN,CALLS
0,1320387639904735235,@elpaisuy Lo gracioso de esto es ver todos los...,2020-10-26 00:27:01.700,0,0,0,0,0,0,0,0,0
1,1369424918572630021,@cronica Qué cachivache ésta 👇 mina😡,2021-03-09 23:13:03.100,0,0,0,0,0,0,0,0,0
2,1260310148058316807,@infobae Es un tarado.. no endurece con los re...,2020-05-12 20:49:05.800,1,0,0,0,0,1,0,0,0
3,1301508996717740034,@pagina12 Como siempre bien informados sobre l...,2020-09-03 14:22:04.200,0,0,0,0,0,0,0,0,0
4,1375042282706112514,@abc_es A este no era al que se le volaron los...,2021-03-25 19:08:01.700,0,0,0,0,0,0,0,0,0


Aplico el modelo con finetuning de BETO MULTIETIQUETA

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

checkpoint = "natdebandi/beto_hate_speech_ar_multiclasif"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

tokens = tokenizer(df_sample['tweet'].tolist(), padding=True, truncation=True, return_tensors="pt")
outputs = model(**tokens)



tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/736k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
outputs[1]

In [ ]:
# prompt: use the transformers librerys of higging face to apply a mult label clasification to the varible tweet in df_sample

!pip install transformers

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the pre-trained model and tokenizer
model_name = "Davlan/bert-base-multilingual-cased-hate-speech-spanish"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Prepare the data for the model
def preprocess_function(examples):
    return tokenizer(examples["tweet"], truncation=True, padding=True)

encoded_dataset = df_sample.map(preprocess_function, batched=True)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Perform inference
predictions = []
with torch.no_grad():
    for i in tqdm(range(len(df_sample))):
        input_ids = torch.tensor(encoded_dataset['input_ids'][i]).unsqueeze(0).to(device)
        attention_mask = torch.tensor(encoded_dataset['attention_mask'][i]).unsqueeze(0).to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.sigmoid(logits).squeeze().tolist()
        predictions.append(predicted_labels)

# Add predictions to the dataframe
df_sample['predictions'] = predictions
